In [ ]:
from qiskit import QuantumCircuit
import random
import itertools
import qiskit_aer

# Parameters
n = 21
bitstrings = [''.join(p) for p in itertools.product('01', repeat=n)]
s = random.choice(bitstrings)  # secret string

# Inner product mod 2
def g(s, x):
    return sum(int(a) & int(b) for a, b in zip(s, x)) % 2

# Bernstein-Vazirani circuit to find s
def bernstein_vazirani_circuit(s):
    n = len(s)
    qc = QuantumCircuit(n + 1, n)

    qc.x(n)       # Set output qubit to |1⟩
    qc.h(n)       # Hadamard on output qubit
    qc.h(range(n))  # Hadamard on input qubits

    # Oracle U_f: flips phase based on s · x
    for i, bit in enumerate(s):
        if bit == '1':
            qc.cx(i, n)

    qc.h(range(n))        # Hadamard again
    qc.measure(range(n), range(n))  # Measure input qubits

    return qc

# Build and run
qc = bernstein_vazirani_circuit(s)
print(qc.draw())
sim = qiskit_aer.AerSimulator()
result = sim.run(qc, shots=1024).result()
counts = result.get_counts()

# Show results
print(f"Secret string used: {s}")
print("Measured result:", counts)